**CHEST X-RAY IMAGE CLASSIFICATION ADVANCED DATA SCIENCE CAPSTONE PROJECT**

Paolo Cavadini, February 2021.

Dataset
https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia

Find on GitHub: https://github.com/pcavad/capstone_x_rays

In [ ]:
import os

import keras
from keras.preprocessing import image
from keras import backend as K
from keras.models import Sequential, load_model
from keras import layers
from keras.layers import Input, Dense, Dropout, Flatten, MaxPool2D 
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, SeparableConv2D 
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint,EarlyStopping 
import itertools
import matplotlib.pyplot as plt
plt.style.use('seaborn')
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score 
import seaborn as sns
import tensorflow as tf

**MODEL EVALUATE**

In [ ]:
model = load_model('./data/model.h5')

In [ ]:
preds = np.load('./data/preds.npy')

In [ ]:
history = pd.read_csv('./data/training_history.csv')

In [ ]:
X_test = np.load('./data/X_test.npy')
y_test = np.load('./data/y_test.npy')

**Evaluating against the test set**

In [ ]:
#Model metrics
score = model.evaluate(X_test, y_test, batch_size=32) # test set
print('Test loss: {:.2f}%'.format(score[0]*100))
print('Test accuracy: {:.2f}%'.format(score[1]*100))

In [ ]:
# Plotting the validation curves.

def plot_validation_curves(result):
    '''
    This function plots the validation curves.
    '''
    result = pd.DataFrame(result)
    fig, axs = plt.subplots(1,2)
    result[['loss','val_loss']].plot(figsize=(10, 3),ax=axs[0])
    axs[0].set_title('Train vs validation Loss')
    axs[0].set_xlabel('Epochs')
    axs[0].set_ylabel('Loss')
    result[['accuracy','val_accuracy']].plot(figsize=(10, 3),ax=axs[1])
    axs[1].set_title('Train vs validation Accuracy')
    axs[1].set_xlabel('Epochs')
    axs[1].set_ylabel('Accuracy')

In [ ]:
plot_validation_curves(history)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], '.0f'),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, np.round(preds), labels=[1,0])
# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['pneumonia','normal'],normalize= False,  title='Confusion matrix')

In [ ]:
# Showing a randomly selected scan along with the probability that it's a pneumonia vs normal.

random_num = np.random.randint(0,len(X_test))
pred = model.predict(X_test[random_num:random_num+1,:,:,:])
score = pred[0]

print('This image is {:.2f}% pneumonia and {:.2f}% normal.'.format(100 * score[0],100*(1-score[0])))
plt.figure(figsize=(12,6))
plt.imshow(X_test[random_num,:,:,:])
plt.grid(False)
plt.axis('off')
plt.show()